# `fuzzup` 
`fuzzup` offers (1) a simple approach for clustering string entitities based on 
[Levenshtein Distance](https://en.wikipedia.org/wiki/Levenshtein_distance) using
[Fuzzy Matching](https://en.wikipedia.org/wiki/Fuzzy_matching_(computer-assisted_translation))
in conjunction with a simple rule-based clustering method. 

`fuzzup` also provides (2) functions for computing the prominence of  
entity clusters resulting from (1).

In this section we will go through the nuts and bolts of `fuzzup` by applying it to a realistic setting.

## Designed for handling output from NER
An important use-case for `fuzzup` is organizing and structuring output from [Named-Entity Recognition](https://en.wikipedia.org/wiki/Named-entity_recognition)(=NER).

For this reason `fuzzup` has been handtailored to fit the output from NER predictions from the [Hugging Face](https://huggingface.co/) [transformers](https://github.com/huggingface/transformers) [NER pipeline](https://huggingface.co/docs/transformers/v4.16.2/en/main_classes/pipelines#transformers.TokenClassificationPipeline) specifically.


## Use-case
First of, import dependencies needed later.

In [33]:
from rapidfuzz.fuzz import partial_token_set_ratio
import pandas as pd
import numpy as np

from fuzzup.fuzz import (
    fuzzy_cluster, 
    compute_prominence, 
    compute_fuzzy_matrix
)

Say, we have used a `transformers` Hugging Face NER pipeline to identify names of persons in a news article. The output from the algorithm is a list of string entities and looks like this (simulated data).

In [34]:
PERSONS = ['Donald Trump', 'Donald Trump', 'J. biden', 'joe biden', 'Biden', 'Bide', 'mark esper', 'Christopher c . miller', 'jim mattis', 'Nancy Pelosi', 'trumps', 'Trump', 'Donald', 'miller']
# align format with output from Hugging Face `transformers` pipeline
n = len(PERSONS)
PERSONS_NER = pd.DataFrame(data = PERSONS, columns=['word'])
PERSONS_NER["entity_group"] = "PER"
PERSONS_NER["score"] = np.random.sample(n)
PERSONS_NER["start"] = np.random.randint(100, size=n)
PERSONS_NER["end"] = np.random.randint(100, size=n)
print(PERSONS_NER)
PERSONS_NER = PERSONS_NER.to_dict(orient="records")

                      word entity_group     score  start  end
0             Donald Trump          PER  0.707755     26   59
1             Donald Trump          PER  0.203877     20   66
2                 J. biden          PER  0.056004     49   32
3                joe biden          PER  0.294578      8   83
4                    Biden          PER  0.579199      3   81
5                     Bide          PER  0.354262     83   74
6               mark esper          PER  0.126862     91   70
7   Christopher c . miller          PER  0.720742     50   51
8               jim mattis          PER  0.182735     31   61
9             Nancy Pelosi          PER  0.177108     91   84
10                  trumps          PER  0.568360     56   94
11                   Trump          PER  0.127025     53   66
12                  Donald          PER  0.176698     86   43
13                  miller          PER  0.606761     88   97


As you can see, the output is rather messy (partly due to the stochastic nature of the algorithm). Another reason for the output looking messy is, that for instance 'Joe Biden' has been mentioned a lot of times but in different ways, e.g. 'Joe Biden', 'J. Biden' and 'Biden'. 

We want to organize these strings entities by forming meaningful clusters from them, in which the entities are closely related based on their pairwise edit distances. 

## Workflow

The solution `fuzzup` offers for this task consists of three steps

1. Compute all of the mutual string distances (Levensteihn Distances/fuzzy ratios) between the strings
2. Form clusters of strings based on the distances from (1)
3. Compute prominence of the clusters based on number of occurrences, their positions in the text etc.

### Step 1: Compute Pairwise Edit Distances

First, `fuzzup` computes pairwise fuzzy ratios for all pairs of string entities.

[Fuzzy ratios](https://en.wikipedia.org/wiki/Fuzzy_matching_(computer-assisted_translation)) are numbers between 0 and 100 are measures of similarity between strings. They are derived from the [Levenshtein distance](https://en.wikipedia.org/wiki/Levenshtein_distance) - a string metric, that measures the distance between two strings. 

In short the Levenshtein distance (also known as 'edit distance') between two words is the minimum number of single-character edits (insertions, deletions or substitutions) required to change one word into the other. 

`fuzzup` has a separate function `compute_fuzzy_matrix` for this, that presents the output - the mutual fuzzy ratios - as a cross-tabular matrix with all ratios. 

In [35]:
from fuzzup.fuzz import fuzzy_cluster
compute_fuzzy_matrix(PERSONS, scorer=partial_token_set_ratio)

,J. biden,Bide,joe biden,mark esper,Nancy Pelosi,Donald Trump,trumps,Donald,jim mattis,Biden,Trump,Christopher c . miller,miller
J. biden,100.000000,75.000000,100.000000,22.222221,26.666666,18.181818,0.000000,28.571428,26.666666,88.888885,0.000000,42.857143,40.000000
Bide,75.000000,100.000000,75.000000,40.000000,40.000000,25.000000,0.000000,40.000000,33.333332,100.000000,0.000000,50.000000,50.000000
joe biden,100.000000,75.000000,100.000000,30.769230,35.294117,25.000000,0.000000,25.000000,30.769230,80.000000,0.000000,33.333332,40.000000
mark esper,22.222221,40.000000,30.769230,100.000000,26.666666,28.571428,33.333332,22.222221,37.500000,33.333332,40.000000,50.000000,40.000000
Nancy Pelosi,26.666666,40.000000,35.294117,26.666666,100.000000,23.529411,25.000000,25.000000,23.529411,33.333332,0.000000,30.000000,28.571428
Donald Trump,18.181818,25.000000,25.000000,28.571428,22.222221,100.000000,80.000000,100.000000,25.000000,25.000000,100.000000,27.272728,33.333332
trumps,0.000000,0.000000,0.000000,33.333332,25.000000,80.000000,100.000000,0.000000,44.444443,0.000000,80.000000,33.333332,28.571428
Donald,28.571428,40.000000,25.000000,22.222221,25.000000,100.000000,0.000000,100.000000,18.181818,33.333332,0.000000,22.222221,22.222221
jim mattis,26.666666,33.333332,30.769230,40.000000,23.529411,25.000000,44.444443,18.181818,100.000000,28.571428,25.000000,35.294117,33.333332
Biden,88.888885,100.000000,80.000000,33.333332,33.333332,25.000000,0.000000,33.333332,28.571428,100.000000,0.000000,40.000000,40.000000


The different string representations of e.g. Donald Trump and Joe Biden have high mutual fuzzy ratios. In comparision representations of different persons have relatively small fuzzy ratios.

You can think of this matrix as a correlation matrix, that shows the correlation between strings.

### Step 2: Forming Clusters
Clusters of entities can be formed using the output from (1) using a naive approach clustering two string entities together, if their mutual fuzzy ratio exceeds a certain threshold.

Computing the pairwise fuzzy ratios and forming the clusters can be done in one take by invoking the `fuzzy_cluster` function.


In [36]:
clusters, _ = fuzzy_cluster(PERSONS_NER, 
                         scorer=partial_token_set_ratio, 
                         cutoff=70,
                         merge_output=True)
pd.DataFrame.from_records(clusters)

,word,entity_group,score,start,end,cluster_id
0,Donald Trump,PER,0.707755,26,59,Donald Trump
1,Donald Trump,PER,0.203877,20,66,Donald Trump
2,J. biden,PER,0.056004,49,32,joe biden
3,joe biden,PER,0.294578,8,83,joe biden
4,Biden,PER,0.579199,3,81,joe biden
5,Bide,PER,0.354262,83,74,joe biden
6,mark esper,PER,0.126862,91,70,mark esper
7,Christopher c . miller,PER,0.720742,50,51,Christopher c . miller
8,jim mattis,PER,0.182735,31,61,jim mattis
9,Nancy Pelosi,PER,0.177108,91,84,Nancy Pelosi


Note, that the original entities are now equipped with a 'cluster_id', assigning each of the entities to an entity cluster.

We see from the results, that different string representations of e.g. 'Donald Trump' have been clustered together. As you see, the 'cluster_id' of each cluster is the longest string within the entity cluster.

In this case we applied a `partial_token_set_ratio` and a cutoff threshold value of 75 on the pairwise fuzzy ratios. Depending on your use case, you should choose an appropriate scorer from `rapidfuzz.fuzz` and 'fine-tune' the cutoff threshold value on your own data.

### Step 3: Rank Clusters
A naïve approach for computing the 'prominence' of the different string clusters is to just count the number of nodes/strings in each cluster. This is the default behaviour of `compute_prominence()`.

In [37]:
compute_prominence(clusters,
                   to_dataframe=True,
                   merge_output=True)

,word,entity_group,score,start,end,cluster_id,prominence_score,prominence_rank
0,Donald Trump,PER,0.707755,26,59,Donald Trump,5.0,1
1,Donald Trump,PER,0.203877,20,66,Donald Trump,5.0,1
2,J. biden,PER,0.056004,49,32,joe biden,4.0,2
3,joe biden,PER,0.294578,8,83,joe biden,4.0,2
4,Biden,PER,0.579199,3,81,joe biden,4.0,2
5,Bide,PER,0.354262,83,74,joe biden,4.0,2
6,mark esper,PER,0.126862,91,70,mark esper,1.0,4
7,Christopher c . miller,PER,0.720742,50,51,Christopher c . miller,2.0,3
8,jim mattis,PER,0.182735,31,61,jim mattis,1.0,4
9,Nancy Pelosi,PER,0.177108,91,84,Nancy Pelosi,1.0,4


In this case, the 'prominence score' of the 'Donald Trump' entity cluster is 5, because Donald Trump is mentioned 5 times in different variations. This is the highest frequency among the clustered and therefore the 'Donald Trump' cluster is scored as the most prominent cluster.

The clusters are ranked by their prominence scores in the 'prominence rank' column.